In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
lst=torch.Tensor([[1,2,3,4],
                [2,3,4,5],
                 [7,8,9,10]])

In [3]:
lst =lst.unsqueeze(1)

In [4]:
lst.shape

torch.Size([3, 1, 4])

In [5]:
multi_atten=nn.MultiheadAttention(embed_dim=4,
                                  num_heads=2)

In [6]:
multi_atten(lst,lst,lst)

(tensor([[[-1.0817,  0.8499,  0.5107, -1.4308]],
 
         [[-1.2068,  0.8185,  0.5901, -1.4901]],
 
         [[-1.7884,  0.5670,  0.5783, -1.3496]]], grad_fn=<AddBackward0>),
 tensor([[[0.1678, 0.1947, 0.6375],
          [0.1247, 0.1502, 0.7251],
          [0.0299, 0.0466, 0.9235]]], grad_fn=<DivBackward0>))

In [3]:
from collections import defaultdict

In [4]:
rec = defaultdict(lambda : 0 ) 

In [7]:
rec1 = defaultdict()

In [8]:
rec['c']

0

In [9]:
rec1['c']

KeyError: 'c'

In [1]:
import collections

In [7]:
c1 = collections.Counter('abcdfgdgdfgdfgdg')
c2 = collections.Counter('dddbca')

In [8]:
b = c1 - c2

In [9]:
print(b)

Counter({'g': 5, 'f': 3, 'd': 2})


In [10]:
b

Counter({'d': 2, 'f': 3, 'g': 5})

In [8]:
def twoSum(nums, target):
    records = dict()
    for index, value in enumerate(nums):  
        if target - value in records:   # 遍历当前元素，并在map中寻找是否有匹配的key
            return [records[target- value], index]
        records[value] = index    # 遍历当前元素，并在map中寻找是否有匹配的key
    print(records)
    return records

In [9]:
nums = [5,4,3,2,1,0,1,1,1]

In [10]:
A = twoSum(nums=nums,target=1)

In [11]:
A

[4, 5]

In [12]:
for i in range(0,10,2):
    print(i)

0
2
4
6
8


In [13]:
s = 'end'
res = list(s)

In [14]:
res

['e', 'n', 'd']

In [15]:
res[0:4]

['e', 'n', 'd']

In [ ]:
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange



class ConvAttentionLayer_Decouple(nn.Module):
    def __init__(self, input_channels: int, numhead: int = 1, reduction_ratio=2):
        r"""
        Args:
            input_channels (int): input channel of conv attention
            numhead (int, optional): the number of attention heads. Defaults to 1.
        """
        super().__init__()
        # self.q_conv = nn.Conv2d(input_channels, input_channels // reduction_ratio, 3, 1, 1)
        self.q_sem_conv = nn.Conv2d(input_channels,
                                    input_channels // reduction_ratio, 3, 1, 1)
        self.q_geo_conv = nn.Conv2d(input_channels,
                                    input_channels // reduction_ratio, 3, 1, 1)
        # self.k_conv = nn.Conv2d(input_channels, input_channels // reduction_ratio, 3, 1, 1)
        self.k_sem_conv = nn.Conv2d(input_channels,
                                    input_channels // reduction_ratio, 3, 1, 1)
        self.k_geo_conv = nn.Conv2d(input_channels,
                                    input_channels // reduction_ratio, 3, 1, 1)
        # self.v_conv = nn.Conv2d(input_channels, input_channels, 3, 1, 1)
        self.v_conv = nn.Conv2d(input_channels, input_channels, 1, 1, 0)
        self.out_sem_conv = nn.Conv2d(input_channels, input_channels, 1, 1)
        self.out_geo_conv = nn.Conv2d(input_channels, input_channels, 1, 1)
        self.softmax = nn.Softmax(dim=-1)
        self.channels = input_channels // reduction_ratio
        self.numhead = numhead
        self.head_dim = self.channels // numhead
        self.sem_norm = nn.LayerNorm(input_channels)
        self.geo_norm = nn.LayerNorm(input_channels)

    def forward(self,
                query: torch.Tensor,
                key: torch.Tensor,
                value: torch.Tensor,
                q_pos_emb: Optional[torch.Tensor] = None,
                k_pos_emb: Optional[torch.Tensor] = None):
        r"""
        Args:
            query (torch.Tensor, [B, C, H_qk, W_qk]): feature of query
            key (torch.Tensor, [B, C, H_qk, W_qk]): feature of key
            value (torch.Tensor, [B, C, H_v, W_v]): feature of value
            q_pos_emb (Optional[torch.Tensor], optional, [[B, C, H_q, W_q]]):
                positional encoding. Defaults to None.
            k_pos_emb (Optional[torch.Tensor], optional, [[B, C, H_kv, W_kv]]):
                positional encoding. Defaults to None.
        """

        view = query + 0  # NOTE: a funny method to deepcopy
        input_channel = view.shape[1]
        if q_pos_emb is not None:
            query += q_pos_emb
        if k_pos_emb is not None:
            key += k_pos_emb

        # to qkv forward
        # q = self.q_conv(query)
        q_sem = self.q_sem_conv(query) # [B,C/reduction_ratio,H_qk,W_qk]
        q_geo = self.q_geo_conv(query) # 
        qs = [q_sem, q_geo]
        # k = self.k_conv(key)
        k_sem = self.k_sem_conv(key) # [B,C/reduction_ratio,H_qk,W_qk]
        k_geo = self.k_geo_conv(key)
        ks = [k_sem, k_geo]
        v = self.v_conv(value)
        vs = [v, v]
        out_convs = [self.out_sem_conv, self.out_geo_conv]
        norms = [self.sem_norm, self.geo_norm]
        outputs = []
        attentions = []

        for q, k, v, out_conv, norm in zip(qs, ks, vs, out_convs, norms):

            '''
            query (torch.Tensor, [B, C/reduction_ratio, H_q, W_q]): feature of query
            key (torch.Tensor, [B, C/reduction_ratio, H_kv, W_kv]): feature of key
            value (torch.Tensor, [B, C, H_v, W_v]): feature of value
            '''

            # read shape of qkv
            bs = q.shape[0]
            qk_channel = q.shape[1]  # equal to the channel of `k`
            v_channel = v.shape[1]  # channel of `v`
            h_q, w_q = q.shape[2:]  # height and weight of query map
            h_kv, w_kv = k.shape[2:]  # height and weight of key and value map
            numhead = self.numhead
            qk_head_dim = qk_channel // numhead
            v_head_dim = v_channel // numhead

            # scale query
            scaling = float(self.head_dim) ** -0.5
            q = q * scaling

            # reshape(sequentialize) qkv
            # B, C, H_qk, W_qk
            q = rearrange(q, "b c h w -> b c (h w)", b=bs, c=qk_channel, h=h_q, w=w_q)  
            q = rearrange(q, "b (n d) (h w) -> (b n) (h w) d", b=bs,
                          n=numhead, h=h_q, w=w_q, d=qk_head_dim)  # b*n,h_q*w_q,qk_head_dim
            q = q.contiguous()
            # B, C, H_qk, W_qk
            k = rearrange(k, "b c h w -> b c (h w)", b=bs, c=qk_channel, h=h_kv, w=w_kv)
            k = rearrange(k, "b (n d) (h w) -> (b n) (h w) d", b=bs,
                          n=numhead, h=h_kv, w=w_kv, d=qk_head_dim) # b*n,h_kv*w_kv,qk_head_dim
            k = k.contiguous()
            v = rearrange(v, "b c h w -> b c (h w)", b=bs, c=v_channel, h=h_kv, w=w_kv)
            v = rearrange(v, "b (n d) (h w) -> (b n) (h w) d", b=bs,
                          n=numhead, h=h_kv, w=w_kv, d=v_head_dim)
            v = v.contiguous()

            # get the attention map
            energy = torch.bmm(q, k.transpose(1, 2))  # [h_q*w_q, h_kv*w_kv]
            attention = F.softmax(energy, dim=-1)  # [h_q*w_q, h_kv*w_kv]
            attentions.append(attention)
            # get the attention output
            r = torch.bmm(attention, v)  # [bs * nhead, h_q*w_q, C']
            r = rearrange(r, "(b n) (h w) d -> b (n d) h w", b=bs,
                          n=numhead, h=h_q, w=w_q, d=v_head_dim)
            r = r.contiguous()
            r = out_conv(r)

            # residual
            temp_view = view + r
            temp_view = temp_view.view(bs, input_channel, -1).permute(2, 0, 1).contiguous() # [B, C, H_qk, W_qk]
            temp_view = norm(temp_view)
            outputs.append(temp_view)
        return outputs, attentions

In [5]:
s= 'we are the hero'
counter = s.count(' ')

In [6]:
counter

3

In [7]:
res = list(s)

In [8]:
res

['w', 'e', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'h', 'e', 'r', 'o']

In [9]:
res.extend([' ']*counter*2)

In [10]:
res

['w',
 'e',
 ' ',
 'a',
 'r',
 'e',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'h',
 'e',
 'r',
 'o',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']